This notebook was used to generate the data shown in Table 3.1 of the manuscript http://arxiv.org/abs/1505.04024.

# Linear programming feasibility problem

We compute the optimal perturbations in 3 ways.  The most reliable way is to use algorithm 1, repeatedly solving the LP given by (3.24).

Here is the problem we solve:
\begin{align} 
 (I-2\alpha^{down}_r)\alpha_r & = (\alpha^{up}_r - \alpha^{down}_r) \\
 (I-2\alpha^{down}_r)v_r & = \gamma_r.
\end{align}
Here $\alpha_r, v_r$ are given and we require $\alpha^{up}_r \ge 0$, $\alpha^{down}_r\ge 0$, $\gamma_r\ge 0$.

We can rewrite the first equation:
\begin{align}
        \alpha_r^d (I+2\alpha_r) - \alpha_r^u = -\alpha_r
\end{align}

To obtain

\begin{align} 
 (I-2\alpha^{down}_r)\alpha_r + \alpha^{down}_r & = (\alpha^{up}_r ) \ge 0 \\
 (I-2\alpha^{down}_r)v_r & = \gamma_r \ge 0.
\end{align}

Alternatively, we can apply Algorithm 2, with either a single iteration or multiple iterations.

All of this is implemented in the Nodepy package, which can be installed with the command

    pip install nodepy

In [1]:
import cvxpy as cvx
from nodepy import rk
from sympy import Matrix
import sympy
import numpy as np
sympy.init_printing()
from colorama import init
from colorama import Fore, Back, Style
init()

As an example, let's compute (approximately) the optimal perturbation for the classical 4-stage, 4th-order method of Kutta:

In [2]:
method = rk.loadRKM('RK44')

The default is to use algorithm 2 with iteration.  In fact, the non-iterative version has been removed from Nodepy as it was only a curiosity.  To run the non-iterative version, you'll need to check out a specific version of Nodepy from Github: https://github.com/ketch/nodepy/commit/38302584dd832fffb848979f77d433fdf5cbd85a.

In [3]:
method.optimal_perturbed_splitting()

(0.6850160627358832,
 array([1.00000000000000, 0.0, 0.0, 0.0, 0.0], dtype=object),
 array([[0, 0, 0, 0, 0],
        [0.671254015683971, 0, 0, 0, 0],
        [0.270090108540258, 0.342508031367942, 0, 0, 0],
        [2.71491162884274e-13, 0, 0.685016062735883, 0, 0],
        [0.306177880587856, 0.176917664597762, 0.150130853210932,
         0.114169343789314, 0]], dtype=object),
 array([[0, 0, 0, 0, 0],
        [0.328745984316029, 0, 0, 0, 0],
        [0.387401860091801, 0, 0, 0, 0],
        [0.0803604341607595, 0.234623503103086, 0, 0, 0],
        [0.252604257814136, 0, 0, 0, 0]], dtype=object))

The meaning of the outputs of the command above can be found by looking at the documentation.  We're mainly interested in the first one, which is the optimal perturbed radius of absolute monotonicity.

The next command computes the same thing, with Algorithm 1:

In [4]:
method.optimal_perturbed_splitting(algorithm='LP')

(0.6850161196809276,
 array([1.00000000000000, 0.0, 0.0, 0.0, 0.0], dtype=object),
 array([[0, 0, 0, 0, 0],
        [0.671254029920232, 0, 0, 0, 0],
        [0.270090084551929, 0.342508059840464, 0, 0, 0],
        [-5.79968717079327e-8, 0, 0.685016119680928, 0, 0],
        [0.306177866582177, 0.176917677257009, 0.150130859189857,
         0.114169353280155, 0]], dtype=object),
 array([[0, 0, 0, 0, 0],
        [0.328745970079768, 0, 0, 0, 0],
        [0.387401855607607, 0, 0, 0, 0],
        [0.0803603962045867, 0.234623542111357, 0, 0, 0],
        [0.252604243690802, 0, 0, 0, 0]], dtype=object))

We see that the values match, to 6 decimal places.

The function below computes bound (3.14) from the paper.

In [5]:
def linear_bound(p,s):
    rad = np.product(np.arange(s-p+1,s+1))
    return rad**(1./p)

In [6]:
def method_stats_str(method):
    method_stats = [method.name,
                method.absolute_monotonicity_radius(),
                0,
                method.optimal_perturbed_splitting(acc=1.e-5,iterate=True)[0],
                method.optimal_perturbed_splitting(acc=1.e-5,iterate=False)[0],
                1./max(np.max(np.abs(method.A)),np.max(np.abs(method.b))),
                linear_bound(method.p,len(method))]
    if method.is_explicit():
        method_stats[2] = method.optimal_perturbed_splitting(algorithm='LP')[0]
    return "%30s %10.3f %10.3f %10.3f %10.3f %10.3f %10.3f" % tuple(method_stats)

## Table of stats for explicit methods

In [7]:
methods = rk.loadRKM()
print Fore.BLUE+"%30s %10s %10s %10s %10s %10s %10s" % ("Method", "SSP coeff", "LP", "Iterate", "No iterate", "1/Amax", "G Lin Bd")
i = 0
for name, method in methods.iteritems():
    if method.is_explicit():
        if i % 2 ==0:
            print Fore.RED  + method_stats_str(method)
        else:
            print Fore.BLACK+Back.YELLOW + method_stats_str(method)
        i += 1

                        Method  SSP coeff         LP    Iterate No iterate     1/Amax   G Lin Bd
                        SSP 75      0.000      1.396      1.396      1.179      1.792      4.789
                       Lambert      0.000      0.238      0.238      0.000      0.583      5.477
              Fehlberg RK5(4)6      0.000      0.058      0.058      0.000      0.125      3.728
                 Forward Euler      1.000      1.000      1.000      1.000      1.000      1.000
                 Merson RK4(3)      0.000      0.243      0.243      0.000      0.500      3.310
                      SSPRK 33      1.000      1.000      1.000      1.000      1.000      1.817
   Minimal Truncation Error 22      0.500      1.000      1.000      1.000      1.333      1.414
           Prince-Dormand 8(7)      0.000      0.013      0.013      0.000      0.060      9.213
                        SSP 95      0.000      2.738      2.738      2.696      3.198      6.853
                 Classical RK4

# End of documented section

The material below this point is left in because it might be useful in the future.  But it is not well documented.

## Table of stats for implicit methods

In [8]:
print Fore.BLUE+"%30s %10s %10s %10s %10s %10s %10s" % ("Method", "SSP coeff", "LP", "Iterate", "No iterate", "1/Amax", "G Lin Bd")
i = 0
for name, method in methods.iteritems():
    if not method.is_explicit():
        if i % 2 ==0:
            print Fore.RED  + method_stats_str(method)
        else:
            print Fore.BLACK+Back.YELLOW + method_stats_str(method)
        i += 1

                        Method  SSP coeff         LP    Iterate No iterate     1/Amax   G Lin Bd
                Implicit Euler        inf      0.000     50.010     50.010      1.000      1.000


KeyboardInterrupt: 

## Checking the column assumption

Seems to hold when my algorithm is applied once:

In [9]:
method = rk.loadRKM('SSP75')

In [10]:
r, gamma, alphaup, alphadown = method.optimal_perturbed_splitting(iterate=False)
inds = np.where(alphaup>1.e-14)
cols_up = set(inds[1])
nz_up = [(inds[0][i],inds[1][i]) for i in range(len(inds[0]))]
inds = np.where(alphadown>1.e-14)
cols_down = set(inds[1])
nz_down = [(inds[0][i],inds[1][i]) for i in range(len(inds[0]))]
print set.intersection(set(nz_up),set(nz_down))
print set.intersection(cols_up,cols_down)
print any(set.intersection(cols_up,cols_down))

set([(7, 0), (3, 0), (6, 0), (2, 0), (5, 0), (1, 0), (4, 0)])
set([0])
False


But not necessarily when my algorithm is iterated:

In [11]:
r, gamma, alphaup, alphadown = method.optimal_perturbed_splitting(iterate=True)
inds = np.where(alphaup>1.e-14)
cols_up = set(inds[1])
nz_up = [(inds[0][i],inds[1][i]) for i in range(len(inds[0]))]
inds = np.where(alphadown>1.e-14)
cols_down = set(inds[1])
nz_down = [(inds[0][i],inds[1][i]) for i in range(len(inds[0]))]
print set.intersection(set(nz_up),set(nz_down))
print set.intersection(cols_up,cols_down)
print any(set.intersection(cols_up,cols_down))

set([(7, 0), (3, 0), (6, 0), (2, 0), (5, 0), (1, 0), (4, 0)])
set([0, 1, 2, 3, 4])
True


Is this a general phenomenon?  It seems not (see below).

# Using Pandas!

In [12]:
import pandas as pd

In [13]:
methods = rk.loadRKM()
df = pd.DataFrame(index=methods.keys(),columns=['method','LP','Iterated','Single pass','SSP Coeff','1/A bound',
                                                'Tihi bound','Column assumption','type'])

In [14]:
df.head()

,method,LP,Iterated,Single pass,SSP Coeff,1/A bound,Tihi bound,Column assumption,type
BE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SSP75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Lambert65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fehlberg45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
for name, method in methods.iteritems():
    df.loc[name]['method'] = method

In [16]:
df.head()

,method,LP,Iterated,Single pass,SSP Coeff,1/A bound,Tihi bound,Column assumption,type
BE,Implicit Euler\n\n 1 | 1\n___|___\n | 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SSP75,SSP 75\nFrom Ruuth-Spiteri paper\n 0.000 |\n 0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Lambert65,Lambert\n\n 0 |\n 1/2 | 1/2\n 1/4 | 1/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fehlberg45,Fehlberg RK5(4)6\n\n 0 |\n 1/4 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FE,Forward Euler\n\n 0 |\n___|___\n | 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
for index, row in df.iterrows():
    method = row['method']
    print index
    if type(method) in (rk.ExplicitRungeKuttaMethod,
                    rk.ExplicitRungeKuttaPair):
        row['LP'] = method.optimal_perturbed_splitting(algorithm='LP')

BE
SSP75
Lambert65
Fehlberg45
FE
Merson43
SSP33
MTE22
PD8
SSP95
RK44
SSP22star
RadauIIA3
RadauIIA2
BS5
Heun33
SSP22
DP5
LobattoIIIC4
NSSP33
NSSP32
SSP85
CMR6
BuRK65
SSP104
LobattoIIIA2
GL2
GL3
LobattoIIIC3
LobattoIIIC2
Mid22


In [18]:
for index, row in df.iterrows():
    method = row['method']
    row['SSP Coeff'] = method.absolute_monotonicity_radius()
    row['1/A bound'] = 1./sympy.N(max(np.max(np.abs(method.A)),np.max(np.abs(method.b))))
    row['Tihi bound'] = linear_bound(method.p,len(method))

In [19]:
for index, row in df.iterrows():
    method = row['method']
    if type(method) in (rk.ExplicitRungeKuttaMethod,
                    rk.ExplicitRungeKuttaPair):    
        row['type'] = 'explicit'
    else:
        row['type'] = 'implicit'

In [20]:
df.head()

,method,LP,Iterated,Single pass,SSP Coeff,1/A bound,Tihi bound,Column assumption,type
BE,Implicit Euler\n\n 1 | 1\n___|___\n | 1,NaN,NaN,NaN,inf,1.00000000000000,1,NaN,implicit
SSP75,SSP 75\nFrom Ruuth-Spiteri paper\n 0.000 |\n 0...,"(1.39601635844, [1.00000000000000, 0.0, 0.0, 0...",NaN,NaN,0,1.79244055420630,4.789389,NaN,explicit
Lambert65,Lambert\n\n 0 |\n 1/2 | 1/2\n 1/4 | 1/...,"(0.237833002627, [1.00000000000000, 0.0, 0.0, ...",NaN,NaN,0,0.583333333333333,5.477226,NaN,explicit
Fehlberg45,Fehlberg RK5(4)6\n\n 0 |\n 1/4 ...,"(0.0578594219191, [1.00000000000000, 0.0, 0.0,...",NaN,NaN,0,0.125000000000000,3.727919,NaN,explicit
FE,Forward Euler\n\n 0 |\n___|___\n | 1,"(1.00000003584, [1.00000000000000, 0.0], [[0, ...",NaN,NaN,1,1.00000000000000,1,NaN,explicit


In [24]:
for index, row in df.iterrows():
    if row['type'] == 'explicit':
        method = row['method']
        r, gamma, alphaup, alphadown = method.optimal_perturbed_splitting(iterate=False)
        row['Single pass'] = r
        inds_up = np.where(alphaup>1.e-14)
        cols_up = set(inds_up[1])
        inds_down = np.where(alphadown>1.e-14)
        cols_down = set(inds_down[1])
        row['Column assumption'] = not any(set.intersection(cols_up,cols_down)) # Assumption satisfied if only column zero is common

In [25]:
df.rename(columns={'Column assumption':'sp Column assumption'},inplace=True)

In [26]:
df['iter Column assumption']=np.nan

In [27]:
for index, row in df.iterrows():
    if row['type'] == 'explicit':
        method = row['method']
        r, gamma, alphaup, alphadown = method.optimal_perturbed_splitting(iterate=True)
        print method.name, r
        df.loc[index,'Iterated'] = r
        inds_up = np.where(alphaup>1.e-14)
        cols_up = set(inds_up[1])
        inds_down = np.where(alphadown>1.e-14)
        cols_down = set(inds_down[1])
        df.loc[index,'iter Column assumption'] = not any(set.intersection(cols_up,cols_down)) # Assumption satisfied if only column zero is common

SSP75 1.39601614193
Lambert65 0.237832940214
Fehlberg RK5(4)6 0.0578594081638
Forward Euler 1.0
Merson RK4(3) 0.242956582456
SSPRK33 1.0
Minimal Truncation Error 22 1.0
SSP95 2.73840349792
Classical RK4 0.685016062736
SSPRK22star 1.21525043702
Bogacki-Shampine RK5(4)8 0.313254463451
Heun33 0.77653792833
SSPRK22 1.0
Dormand-Prince RK5(4)7 0.0407681041854
NSSPRK33 0.558421984903
NSSPRK32 0.77653792833
SSP85 1.87568496164
Calvo 6(5) 0.0215160641728
Butcher's RK65 0.18757842607
Prince-Dormand 8(7) 0.0133671741698
SSPRK(10,4) 6.0
Midpoint Runge-Kutta 0.732050807569


In [28]:
df.head()

,method,LP,Iterated,Single pass,SSP Coeff,1/A bound,Tihi bound,sp Column assumption,type,iter Column assumption
BE,Implicit Euler\n\n 1 | 1\n___|___\n | 1,NaN,NaN,NaN,inf,1.00000000000000,1,NaN,implicit,NaN
SSP75,SSP75\nFrom Ruuth-Spiteri paper\n 0.000 |\n 0....,"(1.39601635844, [1.00000000000000, 0.0, 0.0, 0...",1.396016,1.178508,0,1.79244055420630,4.789389,True,explicit,False
Lambert65,Lambert65\nFrom Shu-Osher paper\n 0 |\n 1/...,"(0.237833002627, [1.00000000000000, 0.0, 0.0, ...",0.2378329,7.756481e-05,0,0.583333333333333,5.477226,False,explicit,False
Fehlberg45,Fehlberg RK5(4)6\n\n 0 |\n 1/4 ...,"(0.0578594219191, [1.00000000000000, 0.0, 0.0,...",0.05785941,1.507554e-07,0,0.125000000000000,3.727919,False,explicit,False
FE,Forward Euler\n\n 0 |\n___|___\n | 1,"(1.00000003584, [1.00000000000000, 0.0], [[0, ...",1,1,1,1.00000000000000,1,True,explicit,True


In [29]:
for index, row in df.iterrows():
    if row['type'] == 'implicit':
        method = row['method']
        r, gamma, alphaup, alphadown = method.__num__().optimal_perturbed_splitting(acc=1.e-5,iterate=False)
        print method.name, r
        df.loc[index,'Single pass'] = r
        inds_up = np.where(alphaup>1.e-14)
        cols_up = set(inds_up[1])
        inds_down = np.where(alphadown>1.e-14)
        cols_down = set(inds_down[1])
        df.loc[index,'sp Column assumption'] = not any(set.intersection(cols_up,cols_down)) # Assumption satisfied if only column zero is common

Implicit Euler 50.0099940383
RadauIIA3 0
RadauIIA2 0
LobattoIIIC4 0
LobattoIIIA2 1.99999866128
Gauss-Legendre RK24 0
Gauss-Legendre RK36 0
LobattoIIIC3 0
LobattoIIIC2 0


In [30]:
for index, row in df.iterrows():
    if row['type'] == 'implicit':
        method = row['method']
        r, gamma, alphaup, alphadown = method.__num__().optimal_perturbed_splitting(acc=1.e-5,iterate=True)
        print method.name, r
        df.loc[index,'Iterated'] = r
        inds_up = np.where(alphaup>1.e-14)
        cols_up = set(inds_up[1])
        inds_down = np.where(alphadown>1.e-14)
        cols_down = set(inds_down[1])
        df.loc[index,'iter Column assumption'] = not any(set.intersection(cols_up,cols_down)) # Assumption satisfied if only column zero is common

Implicit Euler 50.0099940383
RadauIIA3 0.0147968316078
RadauIIA2 0.0184036338329
LobattoIIIC4 0.0106773269176
LobattoIIIA2 1.99999866128
Gauss-Legendre RK24 0.0209671461582
Gauss-Legendre RK36 0.0208240664005
LobattoIIIC3 0.00814958453178
LobattoIIIC2 0.00765476703644


In [31]:
df.head()

,method,LP,Iterated,Single pass,SSP Coeff,1/A bound,Tihi bound,sp Column assumption,type,iter Column assumption
BE,Implicit Euler\n\n 1 | 1\n___|___\n | 1,NaN,50.00999,50.00999,inf,1.00000000000000,1,True,implicit,True
SSP75,SSP75\nFrom Ruuth-Spiteri paper\n 0.000 |\n 0....,"(1.39601635844, [1.00000000000000, 0.0, 0.0, 0...",1.396016,1.178508,0,1.79244055420630,4.789389,True,explicit,False
Lambert65,Lambert65\nFrom Shu-Osher paper\n 0 |\n 1/...,"(0.237833002627, [1.00000000000000, 0.0, 0.0, ...",0.2378329,7.756481e-05,0,0.583333333333333,5.477226,False,explicit,False
Fehlberg45,Fehlberg RK5(4)6\n\n 0 |\n 1/4 ...,"(0.0578594219191, [1.00000000000000, 0.0, 0.0,...",0.05785941,1.507554e-07,0,0.125000000000000,3.727919,False,explicit,False
FE,Forward Euler\n\n 0 |\n___|___\n | 1,"(1.00000003584, [1.00000000000000, 0.0], [[0, ...",1,1,1,1.00000000000000,1,True,explicit,True


In [32]:
sspi3 = rk.SSPIRK3(4)

In [33]:
print sspi3

SSPIRK43

 -sqrt(15)/10 + 1/2 | -sqrt(15)/10 + 1/2
 -sqrt(15)/30 + 1/2 | sqrt(15)/15         -sqrt(15)/10 + 1/2
 sqrt(15)/30 + 1/2  | sqrt(15)/15         sqrt(15)/15         -sqrt(15)/10 + 1/2
 sqrt(15)/10 + 1/2  | sqrt(15)/15         sqrt(15)/15         sqrt(15)/15         -sqrt(15)/10 + 1/2
____________________|________________________________________________________________________________
                    | 1/4                 1/4                 1/4                 1/4


In [34]:
sspi3.absolute_monotonicity_radius()

In [37]:
sspi3.__num__().optimal_perturbed_splitting()

(6.872983346207995,
 array([0.563508326896271, -2.66453525910038e-14, -3.00453925432878e-14,
        1.42068899400378e-27, 0.0317541634481456], dtype=object),
 array([[0.436491673103729, 0, 0, 0, 0],
        [0.563508326896297, 0.436491673103729, 0, 0, 0],
        [-2.32730609900856e-14, 0.563508326896297, 0.436491673103729, 0, 0],
        [-3.00453925432892e-14, -2.32730609900856e-14, 0.563508326896297,
         0.436491673103729, 0],
        [2.13103349100595e-27, -5.15985034193960e-14, -3.99680288865109e-14,
         0.968245836551900, 0]], dtype=object),
 array([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [2.66592267666867e-14, 0, 0, 0, 0],
        [0, 2.66592267666867e-14, 0, 0, 0],
        [-2.44109294898225e-27, 0, 4.57832661529522e-14, 0, 0]], dtype=object))

In [39]:
sspi2 = rk.SSPIRK2(1)
print sspi2

SSPIRK12

 1/2 | 1/2
_____|_____
     | 1


In [40]:
sspi2.absolute_monotonicity_radius()

In [41]:
sspi2.__num__().optimal_perturbed_splitting()

(2.0000000000002727,
 array([0.499999999999966, -6.81676937119846e-14], dtype=object),
 array([[0.500000000000034, 0],
        [1.00000000000007, 0]], dtype=object),
 array([[0, 0],
        [0, 0]], dtype=object))

In [52]:
meths={'sspi3':sspi3,'sspi2':sspi2}

In [53]:
df2 = pd.DataFrame(index=meths.keys(),columns=['method','LP','Iterated','Single pass','SSP Coeff','1/A bound',
                                                'Tihi bound','Column assumption','type'])

In [54]:
df2.head()

,method,LP,Iterated,Single pass,SSP Coeff,1/A bound,Tihi bound,Column assumption,type
sspi2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sspi3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
for name, method in meths.iteritems():
    df2.loc[name]['method'] = method
    
for index, row in df2.iterrows():
    method = row['method']
    row['SSP Coeff'] = method.absolute_monotonicity_radius()
    row['1/A bound'] = 1./sympy.N(max(np.max(np.abs(method.A)),np.max(np.abs(method.b))))
    row['Tihi bound'] = linear_bound(method.p,len(method))

df2.rename(columns={'Column assumption':'sp Column assumption'},inplace=True)
df2['iter Column assumption']=np.nan
for index, row in df2.iterrows():
    method = row['method']
    r, gamma, alphaup, alphadown = method.__num__().optimal_perturbed_splitting(acc=1.e-5,iterate=False)
    df2.loc[index,'Single pass'] = r
    inds_up = np.where(alphaup>1.e-14)
    cols_up = set(inds_up[1])
    inds_down = np.where(alphadown>1.e-14)
    cols_down = set(inds_down[1])
    df2.loc[index,'sp Column assumption'] = not any(set.intersection(cols_up,cols_down)) # Assumption satisfied if only column zero is common
for index, row in df2.iterrows():
    method = row['method']
    print method
    r, gamma, alphaup, alphadown = method.__num__().optimal_perturbed_splitting(acc=1.e-5,iterate=True)
    df2.loc[index,'Iterated'] = r
    inds_up = np.where(alphaup>1.e-14)
    cols_up = set(inds_up[1])
    inds_down = np.where(alphadown>1.e-14)
    cols_down = set(inds_down[1])
    df2.loc[index,'iter Column assumption'] = not any(set.intersection(cols_up,cols_down)) # Assumption satisfied if only column zero is common

SSPIRK12

 1/2 | 1/2
_____|_____
     | 1
SSPIRK43

 -sqrt(15)/10 + 1/2 | -sqrt(15)/10 + 1/2
 -sqrt(15)/30 + 1/2 | sqrt(15)/15         -sqrt(15)/10 + 1/2
 sqrt(15)/30 + 1/2  | sqrt(15)/15         sqrt(15)/15         -sqrt(15)/10 + 1/2
 sqrt(15)/10 + 1/2  | sqrt(15)/15         sqrt(15)/15         sqrt(15)/15         -sqrt(15)/10 + 1/2
____________________|________________________________________________________________________________
                    | 1/4                 1/4                 1/4                 1/4


In [58]:
df2.head()

,method,LP,Iterated,Single pass,SSP Coeff,1/A bound,Tihi bound,sp Column assumption,type,iter Column assumption
sspi2,SSPIRK12\n\n 1/2 | 1/2\n_____|_____\n | 1,NaN,1.999999,1.999999,2,1.00000000000000,0,True,NaN,True
sspi3,SSPIRK43\n\n -sqrt(15)/10 + 1/2 | -sqrt(15)/10...,NaN,6.872979,6.872979,6.872983,3.87298334620742,2.884499,True,NaN,True


In [86]:
df=df.append(df2)
df.loc['sspi2','type']='implicit'
df.loc['sspi3','type']='implicit'

In [90]:
df['name'] = df.index
df = df.drop_duplicates(subset='name')
df.tail()

,1/A bound,Iterated,LP,SSP Coeff,Single pass,Tihi bound,iter Column assumption,method,order,sp Column assumption,stages,type,name
LobattoIIIC3,1.50000000000000,0.008149585,NaN,0,0,0,False,LobattoIIIC3\nThe LobattoIIIC method with 3 st...,4,True,3,implicit,LobattoIIIC3
LobattoIIIC2,2.00000000000000,0.007654767,NaN,0,0,1.414214,False,LobattoIIIC2\nThe LobattoIIIC method with 2 st...,2,True,2,implicit,LobattoIIIC2
Mid22,1.00000000000000,0.7320508,"(0.732050847971, [1.00000000000000, 0.0, 0.0],...",0,0.7320508,1.414214,True,Midpoint Runge-Kutta\n\n 0 |\n 1/2 | 1/2\n__...,2,True,2,explicit,Mid22
sspi2,1.00000000000000,1.999999,NaN,2,1.999999,0,True,SSPIRK12\n\n 1/2 | 1/2\n_____|_____\n | 1,2,True,1,implicit,sspi2
sspi3,3.87298334620742,6.872979,NaN,6.872983,6.872979,2.884499,True,SSPIRK43\n\n -sqrt(15)/10 + 1/2 | -sqrt(15)/10...,3,True,4,implicit,sspi3


## Save to file and write to latex

In [91]:
df_compact = df.drop('method', 1)

In [92]:
df_compact.to_csv('method_properties.csv')

In [93]:
df.to_csv('method_props.csv')

In [94]:
df_compact

,1/A bound,Iterated,LP,SSP Coeff,Single pass,Tihi bound,iter Column assumption,order,sp Column assumption,stages,type,name
BE,1.00000000000000,50.00999,NaN,inf,50.00999,1,True,1,True,1,implicit,BE
SSP75,1.79244055420630,1.396016,"(1.39601635844, [1.00000000000000, 0.0, 0.0, 0...",0,1.178508,4.789389,False,5,True,7,explicit,SSP75
Lambert65,0.583333333333333,0.2378329,"(0.237833002627, [1.00000000000000, 0.0, 0.0, ...",0,7.756481e-05,5.477226,False,2,False,6,explicit,Lambert65
Fehlberg45,0.125000000000000,0.05785941,"(0.0578594219191, [1.00000000000000, 0.0, 0.0,...",0,1.507554e-07,3.727919,False,5,False,6,explicit,Fehlberg45
FE,1.00000000000000,1,"(1.00000003584, [1.00000000000000, 0.0], [[0, ...",1,1,1,True,1,True,1,explicit,FE
Merson43,0.500000000000000,0.2429566,"(0.242956631012, [1.00000000000000, 0.0, 0.0, ...",0,6.324548e-07,3.309751,False,4,False,5,explicit,Merson43
SSP33,1.00000000000000,1,"(1.00000005884, [1.00000000000000, 0.0, 0.0, 0...",1,1,1.817121,True,3,True,3,explicit,SSP33
MTE22,1.33333333333333,1,"(1.00000005178, [1.00000000000000, 0.0, 0.0], ...",0.5,1,1.414214,True,2,True,2,explicit,MTE22
SSP95,3.19809961201033,2.738403,"(2.73840429164, [1.00000000000000, 0.0, 0.0, 0...",0,2.695788,6.853468,False,5,False,9,explicit,SSP95
RK44,1.00000000000000,0.6850161,"(0.685016119681, [1.00000000000000, 0.0, 0.0, ...",0,0.6850161,2.213364,False,4,False,4,explicit,RK44


In [95]:
df_compact.to_json('test.json')

OverflowError: Maximum recursion level reached

In [96]:
df_explicit = df[df['type']=='explicit']
df_forpaper = df_explicit.drop('method', 1)
df_forpaper = df_forpaper.drop('sp Column assumption', 1)
df_forpaper = df_forpaper.drop('Single pass', 1)
df_forpaper = df_forpaper.drop('type', 1)

print df_forpaper.to_latex() #want to do something like float_format = '%10.3f' but it doesn't work

\begin{tabular}{lllllllrrl}
\toprule
{} &           1/A bound &    Iterated &                                                 LP &  SSP Coeff & Tihi bound & iter Column assumption &  order &  stages &        name \\
\midrule
SSP75      &    1.79244055420630 &    1.396016 &  (1.39601635844, [1.00000000000000, 0.0, 0.0, 0... &          0 &   4.789389 &                  False &      5 &       7 &       SSP75 \\
Lambert65  &   0.583333333333333 &   0.2378329 &  (0.237833002627, [1.00000000000000, 0.0, 0.0, ... &          0 &   5.477226 &                  False &      2 &       6 &   Lambert65 \\
Fehlberg45 &   0.125000000000000 &  0.05785941 &  (0.0578594219191, [1.00000000000000, 0.0, 0.0,... &          0 &   3.727919 &                  False &      5 &       6 &  Fehlberg45 \\
FE         &    1.00000000000000 &           1 &  (1.00000003584, [1.00000000000000, 0.0], [[0, ... &          1 &          1 &                   True &      1 &       1 &          FE \\
Merson43   &   0.5000000000

In [97]:
df_implicit = df[df['type']=='implicit']
df_forpaper = df_implicit.drop('method', 1)
df_forpaper = df_forpaper.drop('sp Column assumption', 1)
df_forpaper = df_forpaper.drop('Single pass', 1)
df_forpaper = df_forpaper.drop('type', 1)
df_forpaper = df_forpaper.drop('LP', 1)
df_forpaper = df_forpaper.drop('1/A bound', 1)
df_forpaper = df_forpaper.drop('Tihi bound', 1)

print df_forpaper.to_latex() #want to do something like float_format = '%10.3f' but it doesn't work

\begin{tabular}{llllrrl}
\toprule
{} &     Iterated & SSP Coeff & iter Column assumption &  order &  stages &          name \\
\midrule
BE           &     50.00999 &       inf &                   True &      1 &       1 &            BE \\
RadauIIA3    &   0.01479683 &         0 &                  False &      5 &       3 &     RadauIIA3 \\
RadauIIA2    &   0.01840363 &         0 &                  False &      3 &       2 &     RadauIIA2 \\
LobattoIIIC4 &   0.01067733 &         0 &                  False &      6 &       4 &  LobattoIIIC4 \\
LobattoIIIA2 &     1.999999 &         2 &                   True &      2 &       2 &  LobattoIIIA2 \\
GL2          &   0.02096715 &         0 &                  False &      4 &       2 &           GL2 \\
GL3          &   0.02082407 &         0 &                  False &      6 &       3 &           GL3 \\
LobattoIIIC3 &  0.008149585 &         0 &                  False &      4 &       3 &  LobattoIIIC3 \\
LobattoIIIC2 &  0.007654767 &         0 

In [70]:
df['stages'] = 0
df['order'] = 0

In [72]:
for index, row in df.iterrows():
    method = row['method']
    df.loc[index,'stages'] = len(method)
    df.loc[index,'order'] = method.p

In [73]:
df.head()

,method,LP,Iterated,Single pass,SSP Coeff,1/A bound,Tihi bound,sp Column assumption,type,iter Column assumption,stages,order
BE,Implicit Euler\n\n 1 | 1\n___|___\n | 1,NaN,50.00999,50.00999,inf,1.00000000000000,1,True,implicit,True,1,1
SSP75,SSP75\nFrom Ruuth-Spiteri paper\n 0.000 |\n 0....,"(1.39601635844, [1.00000000000000, 0.0, 0.0, 0...",1.396016,1.178508,0,1.79244055420630,4.789389,True,explicit,False,7,5
Lambert65,Lambert65\nFrom Shu-Osher paper\n 0 |\n 1/...,"(0.237833002627, [1.00000000000000, 0.0, 0.0, ...",0.2378329,7.756481e-05,0,0.583333333333333,5.477226,False,explicit,False,6,2
Fehlberg45,Fehlberg RK5(4)6\n\n 0 |\n 1/4 ...,"(0.0578594219191, [1.00000000000000, 0.0, 0.0,...",0.05785941,1.507554e-07,0,0.125000000000000,3.727919,False,explicit,False,6,5
FE,Forward Euler\n\n 0 |\n___|___\n | 1,"(1.00000003584, [1.00000000000000, 0.0], [[0, ...",1,1,1,1.00000000000000,1,True,explicit,True,1,1
